# Import libraries

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import threading
import time

# Collecting frames from a video 

In [2]:
def collectFrames(videoPath, dictionary):
    cap = cv2.VideoCapture(videoPath)
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        dictionary[i] = frame
        i+=1
    return dictionary

# Multithreading

In [3]:
videoPath1 = 'D:\\From dell\\from E\\headless\\videos\\1.mp4'
videoPath2 = 'D:\\From dell\\from E\\headless\\videos\\2.mp4'
videoPath3 = 'D:\\From dell\\from E\\headless\\videos\\3.mp4'
videoPath4 = 'D:\\From dell\\from E\\headless\\videos\\4.mp4'

framesDict1 = {}
framesDict2 = {}
framesDict3 = {}
framesDict4 = {}

videoPath = [videoPath1, videoPath2, videoPath3, videoPath4]
framesDict = [framesDict1, framesDict2, framesDict3, framesDict4]

In [6]:
start = time.perf_counter()

threads = []

for i in range(4):
    t = threading.Thread(target=collectFrames, args=(videoPath[i], framesDict[i], ))
    t.start()
    threads.append(t)
    
for thread in threads:
    thread.join()

finish = time.perf_counter()

print(f"Finished in {round(finish-start, 2)} seconds (s)")

Finished in 6.98 seconds (s)


# Merging the frames

In [8]:
def getMaxLen(dictionary):
    
    'returns the max number of frames available among the videos'
    
    lengths = []
    for _ in dictionary:
        lengths.append(len(_))
    return np.max(lengths)   

maxFrames = getMaxLen(framesDict)

In [9]:
def fillEmptyFrames(dictionary, maxFrames):
    
    'All the videos are not of the same frame size'
    'returns the frames dictionary after filling out the blank frames, making all the videos of the same frame size'
    
    for i in range(maxFrames):
        if i not in dictionary:
            dictionary[i] = np.zeros((1080,1920,3))
        else:
            pass
    return dictionary

In [12]:
framesDict1 = fillEmptyFrames(framesDict1, maxFrames = maxFrames)
framesDict2 = fillEmptyFrames(framesDict2, maxFrames = maxFrames)
framesDict3 = fillEmptyFrames(framesDict3, maxFrames = maxFrames)
framesDict4 = fillEmptyFrames(framesDict4, maxFrames = maxFrames)

In [13]:
out = cv2.VideoWriter('merged Video_fps 24.avi', cv2.VideoWriter_fourcc(*'DIVX'), 24, (1920,1080)) 
        # fps = minimum of the videos
    
finalFrame = {}

for i in range(maxFrames):   
    row1 = np.hstack((framesDict1[i],framesDict2[i]))
    row2 = np.hstack((framesDict3[i],framesDict4[i]))
    wholeImg = np.vstack((row1, row2))
    finalFrame[i] = cv2.resize(wholeImg, (1920, 1080))
    out.write(finalFrame[i])

out.release()